In [ ]:
# Ver 10.2 Gen 2 
# neural network detecting neighbors 
# Two Q-learning with seperated Q-tables
# Gender added. 
# Transfer q tables to offspring at the same generation.
# mutations in brain are around the origional values. 
import os
import json
import math
import time
import copy
import pygame
import random
import hashlib
import itertools
import numpy as np
from collections import deque
from queue import PriorityQueue
from random import normalvariate    

# Global settings
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
WALL_COLOR = (128, 128, 128)
WALL_COLOR = (128, 128, 128)

class Maze:
    def __init__(self, screen, SCREEN_WIDTH, SCREEN_HEIGHT):
        self.screen = screen
        self.rows, self.columns = 15, 20         
        self.WALL_THICKNESS = 2         
        self.SCREEN_WIDTH = SCREEN_WIDTH
        self.SCREEN_HEIGHT = SCREEN_HEIGHT
        self.wall_width = SCREEN_WIDTH // self.columns
        self.wall_height = SCREEN_HEIGHT // self.rows
        self.walls = []        
        self.exit = None
        self.entrance = None
        
        layout = [
            [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1],
            [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
            [1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0],
            [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
            [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1],
            [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]  ]

        # Create border walls
        for col in range(self.columns):
            self.walls.append(pygame.Rect(col * self.wall_width, 0, self.wall_width, self.WALL_THICKNESS))
            self.walls.append(pygame.Rect(col * self.wall_width, SCREEN_HEIGHT - self.WALL_THICKNESS, self.wall_width, self.WALL_THICKNESS))
        for row in range(self.rows):
            self.walls.append(pygame.Rect(0, row * self.wall_height, self.WALL_THICKNESS, self.wall_height))
            self.walls.append(pygame.Rect(SCREEN_WIDTH - self.WALL_THICKNESS, row * self.wall_height, self.WALL_THICKNESS, self.wall_height))
        # Create internal walls based on maze lay out
        for row, row_data in enumerate(layout):
            for col, cell in enumerate(row_data):
                if cell == 1:
                    wall_rect = pygame.Rect(col * (self.wall_width + self.WALL_THICKNESS),
                                            row * (self.wall_height + self.WALL_THICKNESS),
                                            self.wall_width, self.wall_height)
                    self.walls.append(wall_rect)
        self.create_entrance_and_exit()

    def create_entrance_and_exit(self):
        entrance_row = self.rows - 2
        self.entrance = (self.WALL_THICKNESS, entrance_row * self.wall_height + self.wall_height // 2)
        exit_distance = 1
        exit_row = self.rows - 2
        exit_col = self.columns - exit_distance - 1
        self.exit = (exit_col * self.wall_width + self.wall_width // 2, exit_row * self.wall_height + self.wall_height // 2)
        self.entrance_box = pygame.Rect(self.entrance[0], self.entrance[1], self.WALL_THICKNESS, self.wall_height)
        self.exit_box = pygame.Rect(self.exit[0] - self.WALL_THICKNESS*6, self.exit[1] - self.wall_height//2, self.WALL_THICKNESS*13, 1.3*self.wall_height)
        self.walls = [wall for wall in self.walls if not (self.entrance_box.colliderect(wall) or self.exit_box.colliderect(wall))]

    def draw(self):        
        GREEN = (0, 128, 0)
        RED = (128, 0, 0)
        grouped_walls = []
        for wall in self.walls:
            if grouped_walls and wall.left == grouped_walls[-1][0].right and wall.top == grouped_walls[-1][0].top and wall.height == grouped_walls[-1][0].height:
                grouped_walls[-1].append(wall)
            else:
                grouped_walls.append([wall])
        for group in grouped_walls:
            x = group[0].left
            y = group[0].top
            width = sum(wall.width for wall in group)
            height = group[0].height
            pygame.draw.rect(self.screen, WALL_COLOR, (x, y, width, height))
        pygame.draw.rect(self.screen, GREEN, (self.entrance[0], self.entrance[1], self.WALL_THICKNESS, self.wall_height))
        pygame.draw.ellipse(self.screen, RED, (self.exit[0], self.exit[1], self.WALL_THICKNESS*13, 1.3*self.wall_height))

class Brain:
    def __init__(self ):                                                           
        self.initial_exploration_rate = 0.1         
        self.exploration_rate = self.initial_exploration_rate 
        
        self.last_train_time = time.time()   
        
        replay_memory_size = 10000    
        self.replay_memory = deque(maxlen=replay_memory_size)         
        self.weights = self.precompute_weights() 
        self.q_table1 = {}  # First  Q-table
        self.q_table2 = {}  # Second Q-table
        for state in itertools.product([0, 1], repeat=3):
            self.q_table1[state] = np.zeros(3)
            self.q_table2[state] = np.zeros(3) 
            
    #Neural-network    
    def think(self, state_wall, neighbors): 
        inputs = np.concatenate((state_wall, neighbors)) 
        # Calculate all hidden layers in one go, applying ReLU
        hidden_outputs = np.maximum(0, np.dot(inputs, self.weights['W1']))  # Layer 1
        hidden_outputs = np.maximum(0, np.dot(hidden_outputs, self.weights['W2']))  # Layer 2
        hidden_outputs = np.maximum(0, np.dot(hidden_outputs, self.weights['W3']))  # Layer 3
        output_layer = np.dot(hidden_outputs, self.weights['W4'])
        return np.argmax(output_layer)
    
    #Boltzmann Exploration 
    def decide_direction1(self, state_wall):
        state = state_wall
        q_values = self.q_table1[state]
        temperature = 0.5  # You can adjust this parameter
        probabilities = np.exp(q_values / temperature) / np.sum(np.exp(q_values / temperature))
        action = np.random.choice(range(len(probabilities)), p=probabilities)
        return action
    
    # Q-learning    
    def decide_direction2(self, state_wall):
        min_exploration_rate = 0.03
        exploration_decay_rate = 0.995
        state = state_wall
        if random.random() < self.exploration_rate:
            action = random.randint(0, 2)  # Explore: Choose a random action
        else:
            action = np.argmax(self.q_table2[state])  # Exploit: Choose the best action based on Q-table
        # Decay the exploration rate
        elapsed_time = time.time() - self.last_train_time
        self.exploration_rate = min_exploration_rate + (self.initial_exploration_rate - 
                                min_exploration_rate) * math.exp(-1 * exploration_decay_rate * elapsed_time)
        return action
 
    def update_q_table2(self, state, action, reward, next_state):
        train_interval = 30
        batch_size=32
        learning_rate = 0.001
        discount_factor = 0.9
        """Update Q-table1 with replay memory and training interval."""
        if len(self.replay_memory) < batch_size or time.time() - self.last_train_time < train_interval:
            return
        minibatch = random.sample(self.replay_memory, batch_size)
        self.replay_memory.append((state, action, reward, next_state))
        for state, action, reward, next_state in minibatch:
            old_value = self.q_table2[state][action]
            next_max = np.max(self.q_table2[next_state])
            new_value = (1 -learning_rate) * old_value + learning_rate * (reward +  discount_factor * next_max)
            self.q_table2[state][action] = new_value
        self.last_train_time = time.time() 
        
    def update_q_table1(self, state, action, reward, next_state): 
        learning_rate = 0.001
        discount_factor = 0.9
        old_value = self.q_table1[state][action]
        next_max = np.max(self.q_table1[next_state])
        new_value = (1 - learning_rate) * old_value + learning_rate * (reward +  discount_factor * next_max)
        self.q_table1[state][action] = new_value  
        
    def mutate1(self):
        """Mutate the brain's weights slightly to introduce variation."""
        mutation_prob = 0.1
        for key in self.weights:
            mutation_mask = np.random.rand(*self.weights[key].shape) < mutation_prob
            new_values = np.random.randint(0x00, 0xFF, self.weights[key][mutation_mask].shape) / 255.0
            self.weights[key][mutation_mask] = new_values
            
    def mutate(self):
        """Mutate the brain's weights slightly to introduce variation with a normal distribution."""
        mutation_prob = 0.1
        mutation_std = 0.1  # Standard deviation for the normal distribution
        for key in self.weights:
            mutation_mask = np.random.rand(*self.weights[key].shape) < mutation_prob
            original_values = self.weights[key][mutation_mask] * 255.0
            new_values = np.round(np.random.normal(original_values, mutation_std * 255.0)).astype(int)
            new_values = (new_values % 256).astype(np.uint8)  # Wrap around values using modulo and convert to uint8
            self.weights[key][mutation_mask] = new_values / 255.0

    def precompute_weights(self):
        input_size = 27
        output_size = 3
        hidden_size_1 = 32
        hidden_size_2 = 64     
        hidden_size_3 = 12     
        weights = {
            'W1': np.array([[random.randint(0x00, 0xFF) / 255.0 for _ in range(hidden_size_1)] for _ in range(input_size)]),
            'W2': np.array([[random.randint(0x00, 0xFF) / 255.0 for _ in range(hidden_size_2)] for _ in range(hidden_size_1)]),
            'W3': np.array([[random.randint(0x00, 0xFF) / 255.0 for _ in range(hidden_size_3)] for _ in range(hidden_size_2)]),
            'W4': np.array([[random.randint(0x00, 0xFF) / 255.0 for _ in range(output_size)] for _ in range(hidden_size_3)])
        }        
        return weights        

    def load_weights_from_file(self):
        folder = "saved_creatures_2"
        weight_files = [f for f in os.listdir(folder) if f.endswith(".json")]  # Ensure JSON
        if not weight_files:  # Check if the list is empty
            return None
        file_name = random.choice(weight_files)
        file_path = os.path.join(folder, file_name)

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            # File does not exist or is empty
            return None
        try:
            with open(file_path, 'r') as infile:
                data = json.load(infile)
                weights_as_arrays = data.get('weights')
                if weights_as_arrays is not None:
                    # Convert the loaded lists back to NumPy arrays
                    weights_as_arrays = {k: np.array(v) for k, v in weights_as_arrays.items()}
            # File is automatically closed here, safe to delete
            os.remove(file_path)  # Delete the file after reading
            return weights_as_arrays
        except json.JSONDecodeError as e:
            # Handle the exception if JSON is invalid
            print(f"Error decoding JSON from file {file_name}: {e}")
            return None
        
    def softmax(self, scores):         
        shifted_scores = scores - np.max(scores)# Subtract the maximum score to prevent numerical overflow         
        exponentials = np.exp(shifted_scores) # Exponentiate the shifted scores         
        return exponentials / np.sum(exponentials)# Normalize the exponentials to get probabilities (sum to 1)
        
class Creature:
    def __init__(self, x, y, parent_brain=None ):
        """Initialize a new Creature at coordinates (x, y) with optional parent_brain for inheritance."""
        new_brain_prob = 0.5 # change to 1 for the first run.
        self.x = x
        self.y = y   
        self.gender= random.choice([1,0]) 
        self.size =  5                          # Standard size of the creature
        self.speed = 2                          # Movement speed     
        self.vision_distance = 20               # Visual range of the creature
        self.color = (255, 255, 255)            # Default color: white        
        self.direction = random.randint(0, 360) # Initial random direction       
        # Choose a Brain 
        self.brain = Brain()         
        if parent_brain is None:                     
            if random.random() < new_brain_prob: # 0.5: # probebility of Creating a new Brain
                pass   
            else:                    # load Brain from file               
                self.brain.weights = self.brain.load_weights_from_file()                
        else:                       # inherit Brain           
            self.brain.weights = copy.deepcopy(parent_brain) # Inherit parent's brain  
            
        self.brain.mutate()                             
        self.update_color_based_on_brain()      # Update color based on brain weights                       
        self.total_reward = 0  # Total accumulated reward
        
    def update(self, walls, goal_x, goal_y, population):
        """Update the creature's state based on its environment and actions."""
        # SENSE the sourounding
        current_state = self.sense_walls(walls)                 
        neighbors = self.sense_creatures(population)
        # Decide on the direction using the Brain                     
        if 0.5<random.random():
            action = self.brain.think(current_state, neighbors)
            reward, next_state, new_x, new_y = self.update_pos(action, walls, goal_x, goal_y)
        else:
            if 0.5<random.random():
                action = self.brain.decide_direction2(current_state)
                reward, next_state, new_x, new_y = self.update_pos(action, walls, goal_x, goal_y)
                self.brain.update_q_table2(current_state, action , reward, next_state)      
            else:
                action = self.brain.decide_direction1(current_state)
                reward, next_state, new_x, new_y = self.update_pos(action, walls, goal_x, goal_y)
                self.brain.update_q_table1(current_state, action , reward, next_state)                    
        # Update position if not colliding with walls
        if not any(wall.collidepoint(new_x, new_y) for wall in walls):
            self.x, self.y = new_x, new_y
            
    def update_pos(self, action,walls, goal_x, goal_y):
        # Define a dictionary mapping actions to direction changes
        action_2_direction = {0: -45, 1: 0, 2: 45}
        direction_change = action_2_direction.get(action, 0)  # Default set action to 0
        self.direction += direction_change
        self.direction %= 360   
         # MOVE
        new_x = self.x + math.cos(math.radians(self.direction)) * self.speed
        new_y = self.y + math.sin(math.radians(self.direction)) * self.speed          
        #COLISION
        collision = any(wall.collidepoint(new_x, new_y) for wall in walls)           
        #REWARD
        reward = self.calculate_reward(new_x, new_y, goal_x, goal_y, collision)
        self.total_reward += reward 
        # SENSE the sourounding
        next_state = self.sense_walls(walls)
        return reward, next_state, new_x, new_y
    
    def sense_walls(self, walls):
        """Determine the presence of walls within the creature's field of vision."""
        angles = {'forward': 0, 'left': -45, 'right': 45}  # Directions with associated angles
        wall_sensed = {direction: False for direction in angles}  # Initialize wall detection flags
        # Calculate cosine and sine values for each direction once, to avoid repeated calculations
        for direction, angle in angles.items():
            cos_val = math.cos(math.radians(self.direction + angle))
            sin_val = math.sin(math.radians(self.direction + angle))

            for dist in range(1, self.vision_distance):
                dx = self.x + cos_val * dist
                dy = self.y + sin_val * dist
                point_rect = pygame.Rect(dx, dy, 1, 1)  # Represents the current point being checked

                if any(wall.colliderect(point_rect) for wall in walls):
                    wall_sensed[direction] = True
                    break  # Exit early if a wall is detected in this direction
        return (int(wall_sensed['forward']), int(wall_sensed['left']), int(wall_sensed['right']))


    def sense_creatures(self, others):
            """Determine the presence of creatures within the creature's field of vision."""
            # Preallocate list for distances and indices for sorting
            distances = []
            index_to_creature = {}
            # Only compute necessary attributes once per creature
            for i, other in enumerate(others):
                if other is not self:
                    # Use squared distance to avoid unnecessary square root for comparison
                    squared_distance = (self.x - other.x) ** 2 + (self.y - other.y) ** 2
                    if squared_distance < 100:  # Compare against squared distance to avoid sqrt
                        distances.append(squared_distance)
                        index_to_creature[len(distances) - 1] = other
            # Get indices of the four closest creatures (or fewer) based on sorted distances
            closest_indices = np.argsort(distances)[:4]
            # Extract creature attributes for the closest creatures
            closest_attributes = []
            for idx in closest_indices:
                if idx in index_to_creature:
                    other = index_to_creature[idx]
                    closest_attributes.extend([other.color[0]/255, other.color[1]/255, other.color[2]/255,
                                               other.direction, other.total_reward,int(other.gender == self.gender)])
            # Create a 24-length numpy array filled with zeros
            creatures_array = np.zeros(24)
            # Update the array with the attributes of the closest creatures
            creatures_array[:len(closest_attributes)] = closest_attributes
            return creatures_array
 
    def calculate_reward(self, new_x, new_y, goal_x, goal_y, collision):
        """Calculate the reward based on creature's movement and goal proximity."""
        if collision:
            return -10  # Collision with a wall        
        distance_before = math.sqrt((self.x - goal_x)**2 + (self.y - goal_y)**2)
        distance_after = math.sqrt((new_x - goal_x)**2 + (new_y - goal_y)**2)
        if distance_after < self.size:
            return 100  # Reached the goal
        if distance_after < distance_before:
            return 1  # Moving towards the goal
        else:
            return -1  # Moving away from the goal        
        
    def update_color_based_on_brain(self):        
        r_avg = self.brain.weights['W1'].mean()
        g_avg = self.brain.weights['W2'].mean()
        b_avg = self.brain.weights['W4'].mean()
        # Adjust the red or green average based on gender
        if self.gender == 1:
            r_avg /= 2
        else:
            g_avg /= 2
        # Scale the averages to 0-255 color values and update the color attribute
        self.color = (int(r_avg * 255), int(g_avg * 255), int(b_avg * 255))

    def create_offspring(self, other_parent, birth_place):
        """Generate a new offspring combining the genes (brain weights) of two parents."""                
        offspring_brain = self.brain.weights.copy()
        for key in self.brain.weights:
            shape = self.brain.weights[key].shape
            for i in range(shape[0]):
                for j in range(shape[1]):
                    if random.random() < .5:
                        offspring_brain [key][i, j] = self.brain.weights[key][i, j]
                    else:
                        offspring_brain[key][i, j] = other_parent.brain.weights[key][i, j]                            
        offspring = Creature(birth_place[0], birth_place[1], parent_brain=offspring_brain)
        offspring.update_color_based_on_brain()  # Update color for the new offspring 
        offspring.brain.q_table1= self.brain.q_table1
        offspring.brain.q_table2= self.brain.q_table2
        return offspring
   
        
class PopulationManager:
    def __init__(self, maze, screen, max_creatures):
        self.creatures = []
        self.maze = maze
        self.screen =screen
        self.creatures_reached_exit = []  # Store creatures that have reached the exit
        self.max_creatures = max_creatures
        self.reached_exit_count = 0
        self.offspring_count =0
     
    def update(self):
        # Update creatures and track those reaching the exit.
        for creature in self.creatures[:]:  # Iterate over a shallow copy of self.creatures
            creature.update(self.maze.walls, self.maze.exit[0], self.maze.exit[1], self.creatures)
            if self.has_reached_exit(creature):
                self.save_creature_to_file(creature, self.generate_file_name(creature.brain.weights))
                self.creatures_reached_exit.append(creature)
                self.reached_exit_count += 1
                self.creatures.remove(creature)  # Remove the creature from the list

        # After all creatures have been updated, find the first male and female that reached the exit
        male_reached_exit = next((creature for creature in self.creatures_reached_exit if creature.gender == 0), None)
        female_reached_exit = next((creature for creature in self.creatures_reached_exit if creature.gender == 1), None)

        # Check if both a male and a female have reached the exit
        if male_reached_exit and female_reached_exit:
            #print(f"Male reached: {male_reached_exit}, Female reached: {female_reached_exit}")
            if len(self.creatures) < self.max_creatures:
                new_creature1 = male_reached_exit.create_offspring(female_reached_exit, self.maze.entrance)
                new_creature2 = female_reached_exit.create_offspring(male_reached_exit, self.maze.entrance)
                self.creatures.extend([new_creature1, new_creature2])
                self.offspring_count += 2
                # It is safe to remove creatures from creatures_reached_exit list after new offspring creation
                self.creatures_reached_exit.remove(male_reached_exit)
                self.creatures_reached_exit.remove(female_reached_exit)

    def save_creature_to_file(self, creature, file_name):
            # Convert all NumPy arrays in the weights to lists for JSON serialization
            weights_as_lists = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in creature.brain.weights.items()}
            data_to_save = {
                'weights': weights_as_lists,  # Use the converted weights
            }
            save_path = "saved_creatures_2"
            os.makedirs(save_path, exist_ok=True)
            with open(os.path.join(save_path, file_name), 'w') as outfile:
                json.dump(data_to_save, outfile)  
            
    def generate_file_name(self, weights):
        serializable_weights = {k: v.tolist() if hasattr(v, 'tolist') else v for k, v in weights.items()}
        filename = hashlib.md5(json.dumps(serializable_weights, sort_keys=True).encode('utf-8')).hexdigest()[:10] + '.json'
        return filename

    def spawn_creature(self):
        if len(self.creatures) < self.max_creatures:  # Ensure there are at least two creatures to start with
            x, y = self.maze.entrance
            creature = Creature(x, y)
            creature.direction = 0  # Set initial direction towards the center of the maze
            self.creatures.append(creature)             

    def is_within_bounds(self, creature):
        return (0 <= creature.x <= SCREEN_WIDTH and 0 <= creature.y <= SCREEN_HEIGHT)

    def has_reached_exit(self, creature):
        exit_rect = pygame.Rect(*self.maze.exit, self.maze.WALL_THICKNESS, self.maze.wall_height)  # Create a rect for the exit
        if exit_rect.colliderect(pygame.Rect(creature.x - creature.size, creature.y - creature.size,
                                             creature.size * 2, creature.size * 2)):
            return True
        return False

    def draw(self, font):
        # Draw existing elements (creatures, walls, etc.)
        for creature in self.creatures:
            pygame.draw.circle(self.screen, creature.color, (int(creature.x), int(creature.y)), creature.size)
           
        self.maze.draw()
        # Render the text indicating the number of creatures that reached the exit
        text_surface = font.render(f'Exited: {self.reached_exit_count}', True, (255, 255, 255))  
        self.screen.blit(text_surface, (500, self.maze.SCREEN_HEIGHT - 30))  # Position the text 
        
        text_surface = font.render(f'OffSpring: {self.offspring_count}', True, (255, 255, 255))   
        self.screen.blit(text_surface, (50, self.maze.SCREEN_HEIGHT - 30))  # Position the text
        
class Main:
    def __init__(self):
        self.register_creature = 10
        max_creatures =20
        SCREEN_WIDTH, SCREEN_HEIGHT = 640, 480      
        window_pos_x = 1256        
        window_pos_y = 50
        self.last_spawn_time = time.time()            
        
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        os.environ['SDL_VIDEO_WINDOW_POS'] = f"{window_pos_x},{window_pos_y}"
        pygame.init()
        pygame.display.set_caption('Evolution Simulator')
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont(None, 24)   
        
        self.maze = Maze(self.screen, SCREEN_WIDTH, SCREEN_HEIGHT)  # Adjust rows and columns as needed      
        self.population_manager = PopulationManager(self.maze, self.screen, max_creatures)
        self.creatures_to_spawn = self.population_manager.max_creatures

    def run(self):
        running = True        
        self.start_time = time.time()  # Set the start time
        while running:            
            #spawn initial population
            current_time = time.time()
            if self.creatures_to_spawn > 0 and current_time - self.last_spawn_time >= 1:
                self.population_manager.spawn_creature()
                self.creatures_to_spawn -= 1
                self.last_spawn_time = current_time                        
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
            self.screen.fill(BLACK)
            self.population_manager.update()
            self.population_manager.draw(  self.font)
            pygame.display.flip()
            self.clock.tick(60)            
            # Check if the 10th creature has reached the exit
            if self.population_manager.reached_exit_count >= self.register_creature:
                end_time = time.time()  # Get the end time
                duration = end_time - self.start_time  # Calculate the duration
                current_time = time.strftime("%Y/%m/%d - %I:%M %p", time.localtime(end_time))
                print(f"Date Time: {current_time}; Duration for {self.register_creature} agents to exit: {duration:.2f} seconds")
                running = False
        pygame.quit()

if __name__ == '__main__':
    while True: 
        main = Main()
        main.run()        
        